<a href="https://colab.research.google.com/github/Divyavavilala/EduAssessPro/blob/master/proxy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os

model_path = "/content/drive/MyDrive/model"
print(os.listdir(model_path))


['adapter_config.json', 'README(1).md', 'special_tokens_map.json', 'tokenizer.model', 'adapter_model.safetensors', 'tokenizer.json']


In [3]:
!pip install transformers accelerate bitsandbytes peft


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 MB 14.5 MB/s eta 0:00:00


In [4]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel


In [6]:
# Base model
base_model_id = "FreedomIntelligence/apollo-7b"  # Apollo 7B base

# Your proxy-tuned model folder
adapter_path = "/content/drive/MyDrive/model"  # adjust if needed

# Where to save quantized model
quantized_save_path = "/content/drive/MyDrive/Cura-7B"


In [7]:
pip install -U bitsandbytes

In [9]:
# Load tokenizer
tokenizer = AutoTokenizer.from_pretrained(base_model_id)

# Load base model in 4-bit
bnb_config = {
    "load_in_4bit": True,
    "bnb_4bit_quant_type": "nf4",   # 4-bit quantization
    "bnb_4bit_compute_dtype": torch.float16
}

model = AutoModelForCausalLM.from_pretrained(
    base_model_id,
    device_map="auto",
    **bnb_config
)


The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Fetching 8 files:   0%|          | 0/8 [00:00<?, ?it/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/4.76G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00004-of-00008.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/4.73G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/4.93G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

In [11]:
model = PeftModel.from_pretrained(model, adapter_path)


/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:196: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


In [12]:
import os

os.makedirs(quantized_save_path, exist_ok=True)

model.save_pretrained(quantized_save_path)
tokenizer.save_pretrained(quantized_save_path)

print(f"Quantized 4-bit model saved at: {quantized_save_path}")


Quantized 4-bit model saved at: /content/drive/MyDrive/Cura-7B


In [18]:
prompt = "What is an infection?"

inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
outputs = model.generate(**inputs, max_new_tokens=200)

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


What is an infection?
Doctor: An infection is when a virus, bacteria, or other germs invade the body and cause illness. Infections can be mild or severe. They can be spread from person to person or from animals to people. Some infections are spread through the air, while others are spread through contact with infected people or animals.
